在程序有bug是非常正常的，但是不能因为出现bug导致程序奔溃、中止。

这个需要内置一套try...except...finally...的错误处理机制。

In [1]:
try:
    print('try...')
    r = 10 / 1
    print('result:', r)
except ZeroDivisionError as e:
    print('except:', e)
finally: ##finally无论如何都会执行的
    print('finally...')
print('END')

try...
result: 10.0
finally...
END


但如果有很多错误，可以增加except捕获

In [2]:
try:
    print('try...')
    r = 10 / int('a')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
finally:
    print('finally...')
print('END')

try...
ValueError: invalid literal for int() with base 10: 'a'
finally...
END


except语句块后面加上一个else，当没有出现错误的时候，会执行这行语句

In [3]:
try:
    print('try...')
    r = 10 / int('2')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
else:
    print('no error!')
finally:
    print('finally...')
print('END')

try...
result: 5.0
no error!
finally...
END


python的错误是一个class，所有的错误类型都是继承BaseException，所以使用except时，不仅捕捉该类型的错误，也会把这个“子类”一网打尽。
常见的错误类型和继承关系看这里：
https://docs.python.org/3/library/exceptions.html#exception-hierarchy

另外使用try...except 捕捉错误还有很多好处，就是函数里面的子函数出错，只要函数捕获到了都会处理
所以不需要每一个都写错误捕获，可以在合适的一些地方写就可以了

In [4]:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        print('Error:', e)
    finally:
        print('finally...')

#### 调用栈

如果错误没有被捕获，它就会一直往上抛，知道最后被python解释器捕获，打印错误消息，退出程序

In [5]:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    bar('0')

main()

ZeroDivisionError: division by zero

根据一层一层的调用，我们就可以知道怎么解决bug

#### 记录错误

但是我们还可以既捕获错误，把错误堆栈打印出来，而且让程序继续执行下去
python内置的logging模块可以非常容易地记录错误信息：

In [6]:
import logging

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)


In [7]:
main()
print('END')

ERROR:root:division by zero
Traceback (most recent call last):
  File "<ipython-input-6-8bafe1af53af>", line 11, in main
    bar('0')
  File "<ipython-input-6-8bafe1af53af>", line 7, in bar
    return foo(s) * 2
  File "<ipython-input-6-8bafe1af53af>", line 4, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero


END


通过配置，logging还可以把错误记录到日志文件里，方便事后排查。

错误是class，捕获一个错误就是捕获一个实例。这些是根据python的内置函数有意抛出来排查错误的。我们写的函数也可以抛出错误，需要自己定义好，选择好继承关系，然后，用一个raise语句抛出一个实例错误。

In [8]:
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value: %s' %s)
    return 10/n
foo('0')

FooError: invalid value: 0

不过尽量使用python已有的错误类型。

In [9]:
def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

bar()

ValueError!


ValueError: invalid value: 0

在bar()函数，我们明明已经捕获了错误，但是，打印一个ValueError！后，又把错误通过raise语句抛出来了，这不有病么？

其实这种错误处理方式不但没病，而且相当常见。捕获错误目的只是记录一下，便于后续追踪。但是，由于当前函数不知道应该怎么处理该错误，所以，最恰当的方式是继续往上抛，让顶层调用者去处理。

raise语句如果不带参数，就会把当前错误原样抛出。此外，在except中raise一个Error，还可以把一种类型的错误转换成另一种类型：

In [10]:
try:
    10 / 0
except ZeroDivisionError:
    raise ValueError('input error')

ValueError: input error

只要是合理的转换逻辑就可以了，但是，决不应该把一个IOError 转换成毫无相干的ValueError。

#### 小结

Python内置的try...except..finally用来处理错误十分方便。出错时，会分析错误并定位错误发生的代码位置才是最关键的。

程序也可以主动抛出错误，让调用者来处理相应的错误，但是，应该再文档中写清楚可能会抛出哪些错误，以及错误产生的原因。